# 비디오(Video) 질의 응답 LLM (Gemini)

**주요 흐름**

1. `File API`를 사용하여 비디오 파일을 업로드합니다.
2. `GenerateContent` 요청을 통해 비디오에 대한 질문을 요청합니다.
3. 생성된 응답을 확인합니다.
4. 업로드한 Video 파일을 삭제합니다.


**중요:** 

- 본 튜토리얼의 `File API` 는 인증 및 접근을 위해 `API keys`를 사용합니다. 

- 업로드된 파일은 `API key`의 클라우드 프로젝트와 연결됩니다. 

다른 `Gemini API` 와 달리, `API key`는 `File API`에 업로드한 데이터에 대한 접근 권한도 부여하므로 `API key`를 안전하게 보관하는 데 특별히 주의해야 합니다.

**Reference**

- [Gemini API(Cookbook) - Video](https://ai.google.dev/api/rest/v1/models/generateContent#media)

**API KEY 발급**

- [링크](https://makersuite.google.com/app/apikey?hl=ko) 에서 API KEY를 발급받아주세요.
- 사용자의 Google API 키를 환경 변수 `GOOGLE_API_KEY`로 설정합니다.

`.env` 파일에 아래와 같이 입력합니다.

```
GOOGLE_API_KEY=<사용자의 API KEY>
```


In [2]:
# LangSmith 추적 설정
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름 입력
logging.langsmith("CH04-Gemini-Video")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH04-Gemini-Video


## 비디오 업로드

`Gemini API` 는 비디오 파일 형식을 직접 수용합니다. 

**제한사항**

- `File API`는 2GB 이하의 파일을 수용하며 프로젝트당 최대 20GB의 파일을 저장할 수 있습니다. 
- 파일은 2일 동안 유지되며 API에서 **다운로드할 수 없습니다**. 

본 예제는 테디노트 YouTube 채널에 게시된 비디오를 사용합니다. (다른 비디오로 교체하여 진행해도 좋습니다)

- [🧑‍💻 #PDF 전처리 할 수 있는 건 다 해봤음.](https://youtu.be/O3qFWRObAXw)

In [4]:
# 파일 다운로드 후 teddynote-sample-video.mp4 파일로 저장
!powershell -Command "Invoke-WebRequest -Uri 'https://www.dropbox.com/scl/fi/ugue14fyo010jgc7wuh4g/teddynote-sample-video.mp4?rlkey=wcsiktklt7jgoibsluft3m6z9&st=prv4p2uu&dl=1' -OutFile 'teddynote-sample-video.mp4'"

아래 비디오 파일의 경로를 입력합니다.

In [5]:
# 비디오 파일 이름 지정
video_file_name = "teddynote-sample-video.mp4"

다음으로는 `File API`를 사용하여 비디오 파일을 업로드합니다.

In [8]:
#!pip install google-generativeai
import google.generativeai as genai

# 파일 업로드 진행 메시지 출력
print(f"파일을 업로드 중입니다...")

# 파일 업로드 및 파일 객체 반환
video_file = genai.upload_file(path=video_file_name)

# 업로드 완료 메시지 및 파일 URI 출력
print(f"업로드 완료: {video_file.uri}")

파일을 업로드 중입니다...
업로드 완료: https://generativelanguage.googleapis.com/v1beta/files/te60cu53i6dr


파일을 업로드한 후, `files.get` 을 호출하여 API가 파일을 성공적으로 완료되었는지 확인할 수 있습니다. 

`files.get`은  API 키가 속한 클라우드 프로젝트와 연관된 파일 API에 업로드된 파일을 확인할 수 있게 해줍니다. 

In [9]:
import time

# 비디오 파일 처리 상태 확인
while video_file.state.name == "PROCESSING":
    # 처리 완료 대기 메시지 출력
    print("비디오 업로드 및 전처리가 완료될 때까지 잠시만 기다려주세요...")
    # 10초 대기
    time.sleep(10)
    # 비디오 파일 상태 갱신
    video_file = genai.get_file(video_file.name)

# 처리 실패 시 예외 발생
if video_file.state.name == "FAILED":
    raise ValueError(video_file.state.name)

# 처리 완료 메시지 출력
print(
    f"\n비디오 처리가 완료되었습니다!\n이제 대화를 시작할 수 있어요: " + video_file.uri
)

비디오 업로드 및 전처리가 완료될 때까지 잠시만 기다려주세요...

비디오 처리가 완료되었습니다!
이제 대화를 시작할 수 있어요: https://generativelanguage.googleapis.com/v1beta/files/te60cu53i6dr


비디오가 업로드된 후, `generate_content` 함수를 사용하여 Video 에 대한 질문을 요청할 수 있습니다.

In [10]:
# 프롬프트
prompt = "이 영상에 대해서 짧게 요약해 줄 수 있나요?"

# 모델을 Gemini 1.5 Flash로 설정
model = genai.GenerativeModel(model_name="models/gemini-2.5-flash")

# LLM 답변 요청
response = model.generate_content(
    [prompt, video_file], request_options={"timeout": 600}
)
# 결과 출력
print(response.text)

이 영상은 PDF 문서에서 이미지나 표와 같이 복잡한 레이아웃 요소를 효과적으로 전처리하여 RAG(Retrieval-Augmented Generation) 시스템의 성능을 극대화하는 방법을 설명합니다.

**주요 내용:**

1.  **기존 파싱의 한계:** 일반적인 PDF 텍스트 추출 방식은 이미지나 표, 차트 등의 비텍스트 정보나 문서의 레이아웃 구조(제목, 본문 등)를 제대로 처리하지 못해 RAG 검색 품질이 낮아지는 문제점을 지적합니다.
2.  **고급 전처리 파이프라인 제시:**
    *   **PDF 분할:** 대용량 PDF를 페이지 단위로 분할하여 처리 효율성을 높입니다.
    *   **레이아웃 분석 (Upstage Layout Analyzer 활용):** 문서 내의 텍스트, 이미지, 표 등 각 요소를 정확하게 식별하고, 그 위치(바운딩 박스) 및 HTML 코드와 같은 구조적 정보를 추출합니다.
    *   **요소별 이미지/표 크롭 및 저장:** 추출된 위치 정보를 활용하여 PDF 페이지에서 각 이미지와 표를 개별 PNG 파일로 정교하게 잘라내어 저장합니다.
    *   **텍스트/이미지/표 요약 및 엔티티 추출:** LLM을 사용하여 각 크롭된 이미지와 표의 내용을 텍스트로 요약하거나 핵심 엔티티를 추출합니다. 이는 비텍스트 정보를 검색 가능한 형태로 변환합니다.
    *   **마크다운 변환:** 전처리된 내용을 텍스트, 이미지 경로, 표의 마크다운 형식 등을 포함하여 구조화된 마크다운 파일로 통합합니다.
    *   **`GraphState` 객체 활용:** 이 모든 전처리 과정에서 얻은 정보(원본 파일 경로, 페이지 메타데이터, 각 요소의 타입과 내용, 이미지/표/텍스트 요약본 등)를 `GraphState`라는 단일 파이썬 객체에 효율적으로 저장하여 관리합니다.
3.  **기대 효과:** 이 파이프라인을 통해 LLM에 제공되는 컨텍스트의 품질과 밀도가 크게 향상되어, RAG 시스템이 보다 정확하고 풍부한 답변을 생성할 수 있게

아래는 스트림 출력 예제입니다. (`stream=True` 옵션 추가)


In [11]:
# 프롬프트 생성
prompt = "이 영상에서 Gencon 관련 언급한 부분의 시간을 알려주고, 어떤 내용을 말했는지 알려주세요."

# 모델을 Gemini 1.5 Flash로 설정
model = genai.GenerativeModel(model_name="models/gemini-1.5-flash")

# LLM 스트림 답변 요청
response = model.generate_content(
    [prompt, video_file], request_options={"timeout": 600}, stream=True
)

# 생성된 콘텐츠 출력
for chunk in response:
    print(chunk.text, end="", flush=True)

영상 0:26부터 0:44까지 Gencon에 대해 언급합니다.

Gencon은 2024년 9월 20일 서울에서 열리는 개발자를 위한 무료 AI 컨퍼런스입니다. 강연자와 발표 주제 목록은 Gencon 웹사이트에서 확인할 수 있습니다.  강연 내용은 질문과 답변 세션도 포함되어 있습니다.

## 파일 삭제

파일은 2일 후 자동으로 삭제되거나 `files.delete()`를 사용하여 수동으로 삭제할 수 있습니다.

In [12]:
# 파일 삭제
genai.delete_file(video_file.name)

# 삭제 완료 메시지 출력
print(f"영상을 삭제했습니다: {video_file.uri}")

영상을 삭제했습니다: https://generativelanguage.googleapis.com/v1beta/files/te60cu53i6dr
